# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [6]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [7]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [8]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 84568765,
 'calculationPrice': 'close',
 'change': -0.53,
 'changePercent': -0.00335,
 'close': 171.56,
 'closeSource': 'oiafcfil',
 'closeTime': 1704351106554,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 166.59,
 'delayedPriceTime': 1712768413237,
 'extendedChange': 1.27,
 'extendedChangePercent': 0.00778,
 'extendedPrice': 171,
 'extendedPriceTime': 1709195452879,
 'high': 178.8,
 'highSource': 'etmi1 pnelaedy rcied5u ',
 'highTime': 1651826378293,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 166.06,
 'iexCloseTime': 1693332427391,
 'iexLastUpdated': 1660257165610,
 'iexMarketPercent': 0.015450991085405583,
 'iexOpen': 174.2,
 'iexOpenTime': 1690479043352,
 'iexRealtimePrice': 168.98,
 'iexRealtimeSize': 105,
 'iexVolume': 2346989,
 'lastTradeTime': 1674913927023,
 'latestPrice': 172.88,
 'latestSource': 'Close',
 'latestTime': 'December 1, 2021',
 'latestUpdate': 1718085101950,
 'latestVolume': 155222

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [13]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

14.94

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [14]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [16]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,150.61,47.92,N/A
1,AAL,16.77,-3.3,N/A
2,AAP,228.59,22.74,N/A
3,AAPL,171.68,14.87,N/A
4,ABBV,118.57,28.78,N/A
...,...,...,...,...
500,YUM,129.36,23.88,N/A
501,ZBH,123.00,31.17,N/A
502,ZBRA,582.49,38.27,N/A
503,ZION,62.50,6.1,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [17]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [18]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [19]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [20]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,762.42,3.6,262
1,NRG,36.30,3.69,5509
2,EBAY,67.22,3.72,2975
3,DHI,100.53,4.38,1989
4,MPC,63.77,4.43,3136
5,BEN,33.35,4.63,5997
6,TSN,79.11,4.71,2528
7,LYB,84.78,5.07,2359
8,HOLX,75.29,5.12,2656
9,COF,141.72,5.33,1411


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [21]:
symbol = 'AAPL'
# FINISH batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

KeyError: 'AAPL'

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [22]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [23]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,186.70,N/A,61.63,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,300.26,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,288.00,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,10.05,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,22.10,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.93,N/A,14.97,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,59.86,N/A,32.42,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,33.06,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2923.64,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
209,GRMN,136.09,N/A,22.99,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [24]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
        rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [25]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [26]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe


0      0.829703
1      0.051485
2      0.473267
3       0.29901
4       0.59604
         ...   
500    0.524752
501    0.663366
502    0.746535
503    0.089109
504    0.861386
Name: PE Percentile, Length: 505, dtype: object
0      0.805941
1      0.055446
2      0.588119
3      0.934653
4       0.89703
         ...   
500    0.053465
501    0.294059
502    0.845545
503    0.172277
504    0.942574
Name: PB Percentile, Length: 505, dtype: object
0      0.823762
1      0.047525
2      0.189109
3      0.576238
4      0.571287
         ...   
500    0.751485
501    0.518812
502    0.763366
503    0.463366
504    0.938614
Name: PS Percentile, Length: 505, dtype: object
0      0.861386
1      0.023762
2      0.439604
3       0.39802
4      0.326733
         ...   
500     0.69703
501    0.590099
502    0.815842
503    0.067327
504    0.932673
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      0.861386
1      0.065347
2      0.162376
3      0.665347
4      0.540594
         ...   
5

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,152.63,N/A,47.03,0.829703,9.54,0.805941,7.7100,0.823762,31.266298,0.861386,14.989659,0.861386,N/A
1,AAL,16.44,N/A,-3.34,0.051485,-1.60,0.055446,0.4768,0.047525,-7.209248,0.023762,1.431882,0.065347,N/A
2,AAP,220.08,N/A,22.44,0.473267,4.50,0.588119,1.2800,0.189109,12.663483,0.439604,2.951772,0.162376,N/A
3,AAPL,165.17,N/A,15.23,0.29901,21.65,0.934653,3.7200,0.576238,12.038572,0.39802,9.448647,0.665347,N/A
4,ABBV,116.30,N/A,27.87,0.59604,15.22,0.89703,3.7000,0.571287,10.279035,0.326733,7.493203,0.540594,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.20,N/A,24.44,0.524752,-4.67,0.053465,5.8400,0.751485,20.449872,0.69703,9.519619,0.677228,N/A
501,ZBH,123.28,N/A,31.50,0.663366,1.98,0.294059,3.2600,0.518812,16.940951,0.590099,5.509947,0.380198,N/A
502,ZBRA,587.24,N/A,38.21,0.746535,11.66,0.845545,6.0300,0.763366,27.949571,0.815842,12.053678,0.774257,N/A
503,ZION,65.05,N/A,5.90,0.089109,1.39,0.172277,2.8400,0.463366,4.611493,0.067327,2.558216,0.132673,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [27]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,152.63,N/A,47.03,0.829703,9.54,0.805941,7.7100,0.823762,31.266298,0.861386,14.989659,0.861386,0.836436
1,AAL,16.44,N/A,-3.34,0.051485,-1.60,0.055446,0.4768,0.047525,-7.209248,0.023762,1.431882,0.065347,0.048713
2,AAP,220.08,N/A,22.44,0.473267,4.50,0.588119,1.2800,0.189109,12.663483,0.439604,2.951772,0.162376,0.370495
3,AAPL,165.17,N/A,15.23,0.29901,21.65,0.934653,3.7200,0.576238,12.038572,0.39802,9.448647,0.665347,0.574653
4,ABBV,116.30,N/A,27.87,0.59604,15.22,0.89703,3.7000,0.571287,10.279035,0.326733,7.493203,0.540594,0.586337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.20,N/A,24.44,0.524752,-4.67,0.053465,5.8400,0.751485,20.449872,0.69703,9.519619,0.677228,0.540792
501,ZBH,123.28,N/A,31.50,0.663366,1.98,0.294059,3.2600,0.518812,16.940951,0.590099,5.509947,0.380198,0.489307
502,ZBRA,587.24,N/A,38.21,0.746535,11.66,0.845545,6.0300,0.763366,27.949571,0.815842,12.053678,0.774257,0.789109
503,ZION,65.05,N/A,5.90,0.089109,1.39,0.172277,2.8400,0.463366,4.611493,0.067327,2.558216,0.132673,0.18495


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [28]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [29]:
portfolio_input()

Enter the value of your portfolio:1000000


In [30]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.19,862,5.86,0.085149,0.4258,0.057426,0.3565,0.027723,2.851481,0.031683,0.366214,0.007921,0.04198
1,AAL,16.44,1216,-3.34,0.051485,-1.6000,0.055446,0.4768,0.047525,-7.209248,0.023762,1.431882,0.065347,0.048713
2,PRU,103.35,193,5.68,0.083168,0.6290,0.065347,0.5730,0.051485,4.312652,0.057426,0.570110,0.011881,0.053861
3,BEN,33.36,599,4.46,0.072277,0.7413,0.079208,1.0094,0.138614,3.020549,0.033663,0.796732,0.019802,0.068713
4,AIG,53.83,371,8.21,0.132673,0.7093,0.071287,0.9388,0.124752,4.170602,0.051485,0.921150,0.029703,0.08198
5,TSN,80.12,249,4.69,0.075248,0.8119,0.085149,0.3112,0.017822,3.773245,0.041584,3.225500,0.192079,0.082376
6,BA,193.12,103,-13.10,0.041584,-8.2500,0.045545,1.9200,0.316832,-31.418657,0.005941,-1307.654128,0.00198,0.082376
7,DHI,98.33,203,4.30,0.069307,1.2300,0.131683,0.6339,0.059406,3.338102,0.037624,2.277424,0.114851,0.082574
8,ALL,109.06,183,9.90,0.172277,1.2700,0.142574,0.6552,0.067327,2.248661,0.029703,0.612623,0.013861,0.085149
9,MET,58.11,344,9.73,0.170297,0.7153,0.073267,0.7322,0.087129,4.984296,0.081188,0.729669,0.017822,0.085941


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [31]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [32]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


In [35]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [36]:
writer.save()